In [1]:
from cv import *
from helpers import *
import scipy.sparse as sp
import pandas as pd

In [2]:
ratings = load_data('data_train.csv')

In [3]:
split = sp.find(ratings)

In [4]:
df = pd.DataFrame({'Movie':split[0], 'User': split[1], 'Prediction':split[2]})
df = df[['Movie', 'User', 'Prediction']]
df.head()

,Movie,User,Prediction
0,43,0,4.0
1,60,0,3.0
2,66,0,4.0
3,71,0,3.0
4,85,0,5.0


In [ ]:
cross_validationALS_demo(df)

In [ ]:
import numpy as np
from cost import *
from predictionAlgorithms import *
from helpers import df_to_sparse

In [ ]:
def cross_validationALS(data, k_indices, k, rank, lambda_, numIterations):
    """return the loss of Alternating Least Squares with Regularisation."""
    # get k'th subgroup in test, others in train
    te_indice = k_indices[k]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    train = data.loc[tr_indice]
    test = data.loc[te_indice]
    test.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
    print("train : {s} elements".format(s = train.shape[0]))
    print("test : {s} elements".format(s = test.shape[0]))
    
    print(test.shape[0])
    # Train ALS model
    predictions = ALSPyspark(train, test, rank, lambda_, numIterations)
    print(len(predictions))
    
    # calculate the loss for train and test data
    loss_te = compute_cost(predictions, test.Prediction.values)
    return loss_te

In [5]:
data = df
seed = 1
k_fold = 4
numIterations = 20
rank = 6
lambdas = np.linspace(0.07, 0.08, 3)
# split data in k fold
k_indices = build_k_indices(data, k_fold, seed)
# define lists to store the loss of training data and test data
#rmse_tr = []
mse_te = []
best_lambda = lambdas[0]
best_mse = 10
# cross validation
print("{k}-fold cross validation".format(k = k_fold))
for lambda_ in lambdas:
    #rmse_tr_tmp = []
    mse_te_tmp = []
    print("lambda = {l}".format(l=lambda_))
    for k in range(k_fold):
        #loss_te = cross_validationALS(data, k_indices, k, rank, lambda_, numIterations)
        
        
        # FUNCTION
        te_indice = k_indices[k]
        tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
        tr_indice = tr_indice.reshape(-1)
        train = data.loc[tr_indice]
        test = data.loc[te_indice]
        test.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
        print("train : {s} elements".format(s = train.shape[0]))
        print("test : {s} elements".format(s = test.shape[0]))

        print(test.shape[0])
        # Train ALS model
        predictions = ALSPyspark(train, test, rank, lambda_, numIterations)
        print(len(predictions))

        # calculate the loss for train and test data
        loss_te = compute_cost(predictions, test.Prediction.values)
        
        
        
        
        print("k = {k} : loss = {l}".format(k=k, l = loss_te))
        #rmse_tr_tmp.append(loss_tr)
        mse_te_tmp.append(loss_te)
    #rmse_tr.append(np.mean(rmse_tr_tmp))
    mean_mse = np.mean(mse_te_tmp)
    print(mse_te_tmp)
    print("lambda = {l} : mse = {m}".format(l = lambda_, m = mean_mse))
    if mean_mse < best_mse:
        best_lambda = lambda_
    mse_te.append(mean_mse)

4-fold cross validation
lambda = 0.07
train : 882714 elements
test : 294238 elements
294238
28643


ValueError: operands could not be broadcast together with shapes (28643,) (294238,) 

In [6]:
print(len(predictions))

test.shape

28643


(294238, 3)

In [ ]:
numIterations

In [ ]:
predictions = ALSPyspark(train, test, rank, lambda_, numIterations)

In [ ]:
print(len(predictions))

test.shape

In [8]:
# 1. Format the data into RDD for Spark
pDF = sql_sc.createDataFrame(train)
spDF = pDF.rdd

# 2. Train the model
model = ALS.train(spDF, rank, numIterations,lambda_)

# 3. Perform the prediction on the test data
pDF_test = sql_sc.createDataFrame(test[['User', 'Movie']])
spDF_test = pDF_test.rdd
testdata = spDF_test.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: (r[0], r[1], r[2]))

# 4. Format the predictions before returning them. The predictions must sorted because Spark shuffled them during the predictions
predictions_df = predictions.toDF()
predictions_pd = predictions_df.toPandas()
predictions_pd.sort_values(['_1','_2'],ascending=[1,1],inplace=True)
predic_end = predictions_pd.reset_index(drop=True)

AssertionError: user_product should be RDD of (user, product)

In [ ]:
test[['User', 'Movie']].values

In [ ]:
pDF_test

In [ ]:
predictions.count()
testdata.count()
model.userFeatures()

In [ ]:
pDF_test.show()

In [ ]:
pDF_test.count()

In [7]:
predictions2 = model.predictAll(testdata.values)

NameError: name 'model' is not defined

In [ ]:
spDF.count()